<a href="https://colab.research.google.com/github/9-coding/PyTorch/blob/main/26-loss_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loss Functions
인공지능 모델이 data와 얼마나 잘 맞는지 측정하는 function.
- loss 값은 function에 따라 달라질 수 있음.
- 인공지능 학습은 loss를 최소화하는 방향으로 진행.

Binary Classification
- BCELoss or BCEWithLogitsLoss

Multiple-class Classification:
- CrossEntropyLoss
- LogSoftmax + NLLLoss

Multi-label Classification:
- sigmoid + BCELoss
- MultiLablelSoftMarginLoss

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Binary classification

## BCELoss
binary classification에 사용.
- model의 출력이 0과 1 사이 값이어야 하기 때문에 출력으로 sigmoid 사용.
- pred: sigmoid
- label: 0과 1 사이의 실제 라벨값

In [2]:
# 예측값과 실제값
outputs = torch.tensor([0.7, 0.2, 0.9], requires_grad=True).float()
targets = torch.tensor([1.0, 0.0, 1.0]).float()

# BCELoss 선언
criterion = nn.BCELoss()

# 시그모이드 활성화 적용
outputs = torch.sigmoid(outputs)

# 손실 계산
loss = criterion(outputs, targets)
print('BCELoss:', loss.item())

BCELoss: 0.514159619808197


## BCEWithLogitsLoss
- BCELoss의 variation.
- 내부적으로 sigmoid를 결합시켰으므로 입력으로 logit score를 받을 수 있음.
- pred: sigmoid의 입력값인 **logit score**
- label: 0과 1 사이의 실제 라벨 값

In [3]:
# 원시 출력값 (시그모이드 활성화 적용 전 raw score)
raw_outputs = torch.tensor([0.5, -1.0, 2.0], requires_grad=True)
targets = torch.tensor([1.0, 0.0, 1.0])

# BCEWithLogitsLoss 선언
criterion = nn.BCEWithLogitsLoss()

# 손실 계산 (시그모이드 내부 적용)
loss = criterion(raw_outputs, targets)
print(f'BCEWithLogitsLoss: {loss.item():.4f}')

BCEWithLogitsLoss: 0.3048


## Multi-class classification

## CrossEntropyLoss
내부적으로 softmax 함수 적용.
- input: logit score vector
- output: class index. `dtype=long(int64)`


In [4]:
# 원시 출력값 (softmax 활성화 적용 전)
raw_outputs = torch.tensor(
	[[1.0, 2.0, 3.0],
         [1.0, 2.0, 0.0],
         [0.0, 2.0, 1.0]],
        requires_grad=True).float()
targets = torch.tensor([2, 0, 1]).long()  # 각 샘플의 클래스 인덱스
print(targets.dtype)
# CrossEntropyLoss 선언
criterion = nn.CrossEntropyLoss()

# 손실 계산 (softmax 내부 적용)
loss = criterion(raw_outputs, targets)
print(f'CrossEntropyLoss: {loss.item():.2f}')

torch.int64
CrossEntropyLoss: 0.74


## NLLLoss
Negative Log Likelihood Loss
- 주로 LogSoftmax와 함께 사용됨.
- input: log 확률
- output: class index

In [5]:
# 모델 출력
logits = torch.tensor(
	[[0.1, 0.2, 0.7], # max idx 2
     	 [0.8, 0.1, 0.1], # max idx 0
     	 [0.3, 0.5, 0.2]], # max idx 1
    	requires_grad=True).float()

# 타겟 클래스 인덱스
targets = torch.tensor([2, 0, 1]).long()

# 로그 소프트맥스 적용
log_softmax = F.log_softmax(logits, dim=1)

# NLLLoss 선언
criterion = nn.NLLLoss()
loss = criterion(log_softmax, targets)
print(f'NLLLoss with LogSoftmax: {loss.item():.2f}')

NLLLoss with LogSoftmax: 0.80


# Multiple-label Classification

## MultiLabelSoftMarginLoss
### Multi-label Classification을 위한 loss function.
- 하나의 sample이 여러 개의 class에 동시에 속할 수 있는 classification.
- 각 클래스 레이블을 각각 독립적인 binary classification으로 취급.
- input: model의 logit vector output(클래스 각각의 예측값)
- 내부적으로 logistic function을 적용하여 이를 [0, 1] 범위의 확률로 변환.
- 이 변환된 확률과 실제 레이블의 값을 사용해 binary cross entropy loss 계산해 합침.

### Formula
$$L(y, \hat y)
 = -\cfrac1C \displaystyle \sum^C_{i=1} [y_i \cdot \log(\sigma(\hat y_i)) + (1-y_i)\cdot \log(1-\sigma(\hat y_i))]$$

- 𝜎: logistic function (~sigma)
- 𝑦̂_𝑖: predicted logit score.
- 𝑦_𝑖: label or target.

### Class weight for imbalanced class
각 클래스에 가중치를 부여할 수 있음
- 특정 클래스가 다른 클래스보다 중요
- 데이터셋에서 클래스 불균형을 해소할 필요가 있을 때 유용

In [6]:
logits = torch.tensor([[0.5, -1.0, 3.0], [1.5, -2.0, 0.0]], requires_grad=True)
labels = torch.tensor([[1, 0, 1], [0, 1, 0]], dtype=torch.float)

weights = torch.tensor([0.5, 2.0, 1.5], dtype=torch.float) # class 별 중요도

loss_func = nn.MultiLabelSoftMarginLoss(weight=weights)

loss = loss_func(logits, labels)
print("Loss:", loss.item())

Loss: 1.1801210641860962


## Sigmoid+BCELoss와의 차이점
| | MultiLabelSoftMarginLoss|BCELoss|
|---|---|---|
|sigmoid|내부|외부에서 필요|
|classification|multi-label|binary|
|input|logit|probability|
